In [11]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [12]:
# Hyper Parameters
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [13]:
# MNISt dataset (images and labels)
train_dataset = dsets.MNIST(root='/home/damvantai/Documents/data/',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

In [14]:
test_dataset = dsets.MNIST(root='/home/damvantai/Documents/data/',
                           train=True,
                           transform=transforms.ToTensor())

In [15]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                          shuffle=True)

In [16]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [17]:
# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [18]:
model = LogisticRegression(input_size, num_classes)

In [19]:
# Loss and Optimizer
# Softmax is internally computed
# Set parameters to be updated
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [25]:
# Training the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad() # zero the gradient buffer
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], loss: %.4f' %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
            
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28 * 28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

Epoch [1/5], Step [100/600], loss: 0.6773
Epoch [1/5], Step [200/600], loss: 0.7010
Epoch [1/5], Step [300/600], loss: 0.6205
Epoch [1/5], Step [400/600], loss: 0.6988
Epoch [1/5], Step [500/600], loss: 0.6325
Epoch [1/5], Step [600/600], loss: 0.9103
Epoch [2/5], Step [100/600], loss: 0.7580
Epoch [2/5], Step [200/600], loss: 0.7151
Epoch [2/5], Step [300/600], loss: 0.7346
Epoch [2/5], Step [400/600], loss: 0.7200
Epoch [2/5], Step [500/600], loss: 0.7364
Epoch [2/5], Step [600/600], loss: 0.6770
Epoch [3/5], Step [100/600], loss: 0.7117
Epoch [3/5], Step [200/600], loss: 0.6188
Epoch [3/5], Step [300/600], loss: 0.6321
Epoch [3/5], Step [400/600], loss: 0.6786
Epoch [3/5], Step [500/600], loss: 0.7051
Epoch [3/5], Step [600/600], loss: 0.6767
Epoch [4/5], Step [100/600], loss: 0.8250
Epoch [4/5], Step [200/600], loss: 0.7685
Epoch [4/5], Step [300/600], loss: 0.7362
Epoch [4/5], Step [400/600], loss: 0.5326
Epoch [4/5], Step [500/600], loss: 0.5386
Epoch [4/5], Step [600/600], loss:

In [26]:
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the model on the 10000 test images: 85 %


In [27]:
torch.save(model.state_dict(), 'model.pkl')